# Week 14 Exercises


## Ex1: Basic Text Classification
In this exercise you must implement a basic text classification pipeline and apply it to SMS spam classification.
The pipeline is

* Clean strings and split them into words.
    You need to complete the implementation of **standardize_strings**, **split_strings**.
* Make vocabulary/dictionary with mappings from word to index and index to word and transforming vectors.
    You need to complete the implementation of **make_vocabulary_and_index**, **words_to_vectors**
* Apply a standard Logistic Regression classifier on given data and output the results.
  You need to complete the implementation of **run_bow_classifier**

There is a lot of code and hopefully helpful comments.

If the technicalities of this becomes to hard you can move on to the next exercise where we use a standard implementation from sklearn in python to steps 1 and 2 as well.


In [1]:
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


def get_data():
    #wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
    with open('smsspamcollection/SMSSpamCollection', 'r') as f:
        dat = f.readlines()
    labels = [x.split('\t')[0] for x in dat]
    texts = [x.split('\t')[1] for x in dat]
    sl = set(labels)
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

def test_vectorize():
    """ 
    test words_to_vectors 
    """

    alphabet = 'abcdefghijklmnopqrstuvwxz'
    vocab = {x for x in alphabet}
    index_map = {x: i for (i, x) in enumerate(sorted(vocab))}
    a = [1, 2]
    b = [5]
    c = [0, 10]
    index_lists = [a, b, c]
    word_lists = [[alphabet[i] for i in x] for x in index_lists]
    
    print("TESTING 'make_vocabulary_and_index': \t\t", end='')
    learned_vocab, word_to_index, index_to_word = make_vocabulary_and_index(word_lists)
    target_vocab = {'a', 'b', 'c', 'f', 'k'}    
    
    assert target_vocab == learned_vocab, "Error with 'make_vocabulary_and_index'.\nExpected vocabulary: \t{0}\n Got: \t\t{1}".format(target_vocab, learned_vocab)
    # Test word_to_index and index_to_word by checking they are inverse mappings. 
    # inv_map = {v: k for k, v in tc.word_to_index.items()} 
    # assert tc.index_to_word == inv_map, "Error with 'make_vocabulary_and_index'.\nExpected that the dictionaries 'index_to_word' and 'word_to_index' are each others inverse. This was not the case. "
    print("PASSED!")

    print("TESTING 'words_to_vector': \t\t\t", end='')
    string_vectors = words_to_vectors(word_lists, vocabulary=vocab, index_map=word_to_index)
    def my_idx(sid, wid):
        return word_to_index[word_lists[sid][wid]]
    target = np.zeros((3, len(vocab)))
    target[0, my_idx(0, 0)] = 1
    target[0, my_idx(0, 1)] = 1
    target[1, my_idx(1, 0)] = 1
    target[2, my_idx(2, 0)] = 1
    target[2, my_idx(2, 1)] = 1
    assert np.allclose(string_vectors, target),  "Error with words to vectors"
    
    # print(string_vectors-target)
    print('PASSED!')

def test_string_cleaning():
    """ 
    Test string cleaning
    """
    strings = ['i THINK machine LEARNING is cool.,-()', 'he likes cake cake cake',
               'pandora has a box that should not be opened']

    bad_words = {'i', 'the', 'has', 'he', 'has', 'a', 'is'}
    result = [['think', 'machine', 'learning', 'cool'], ['likes', 'cake', 'cake', 'cake'],
              ['pandora', 'box', 'that', 'should', 'not', 'be', 'opened']]

    print("TESTING 'standardize_strings': \t\t\t", end='')
    expect = ['i think machine learning is cool', 'he likes cake cake cake', 'pandora has a box that should not be opened']
    strings_cleaned = standardize_strings(strings) # lower string remove special characters
    assert strings_cleaned == expect, "Error in 'standardize_strings'.\nExpected \t{0}\nGot \t\t{1}".format(expect, strings_cleaned)
    print('PASSED!')

    print("TESTING 'split_strings': \t\t\t", end='')
    expect = [['i', 'think', 'machine', 'learning', 'is', 'cool'], ['he', 'likes', 'cake', 'cake', 'cake'], ['pandora', 'has', 'a', 'box', 'that', 'should', 'not', 'be', 'opened']]
    word_lists = split_strings(strings_cleaned)
    assert word_lists == expect, "Error in 'split_strings': \nExpected\t{0}\nGot \t\t{1}".format(expect, word_lists)
    print("PASSED!")
    
    
def standardize_strings(string_list):
    """ 
    Standardize strings by 
        1. making them lower case (example 'LaTeX' -> 'latex')
        2. remove any non-alphabetic characters, i.e. 
           characters not in [a-z]. (example "l33t_h@x" -> "lthx")

    For example:
        >>> standardize_strings(['remove . please', 'also .,-_()'])
        ['remove  please', 'also ']
    
    The following methods might be useful: 
        str.lower,
        re.sub (regular expression from re package), 
        str.replace, 


    You can ignore irrelevant spacing since we'll take care of it later. 

    Args:
    string_list: list of strings
    
    Returns:
    list of strings without non-alphabetic characters
    """
    res = []
    
    ### YOUR CODE 1-3 lines
    lower = [x.lower() for x in string_list]
    res = [re.sub('[^a-z ]', '', x)      for x in lower]
    ### END CODE
    assert len(string_list) == len(res)
    return res

def split_strings(strings):
    """ 
    Split a list of strings into list of list of words. The splitting should be done on space ' '.

    For example: 
        >>> split_strings(['split me please', 'me to'])
        [['split', 'me', 'please'], ['me', 'to']]
        
    Try to use list comprehension instead of writing function with loops or recursion. 
    List comprehension works as follows

        >>> dat = list(range(6)) 
        >>> dat
        [0,1,2,3,4,5]

        >>> dat_squared = [x**2 for x in dat] 
        >>> dat_squared
        [0, 1, 4, 9, 16, 25]

    The following method might be useful:
        - str.split()  (for help with splitting strings)

    Args:
    strings: list of strings

    Returns:
    list of lists of strings (words)
    """
    word_lists = []
    
    ### YOUR CODE 
    word_lists = [[x for x in string.split(' ')] for string in strings]
    ### END CODE
    
    assert len(word_lists) == len(strings)
    return word_lists



def make_vocabulary_and_index(word_lists):
    """
    Compute the vocabulary (set), word_to_index (dict), and index_to_word
    for the classifier.
     - vocabulary must be a set with all words in the list of lists of words word_lists
     - word_to_index is a dictionary that maps each word to a unique index
     - index_to_word is a dictionary which is the inverse of word_to_index

    Use set and dict comprehension  (like list comprehension) to _ each with one list of code
    {x for x in [1,2,3,4,5,6]} is a set comprehension which makes a set of elements in [1,2,3,4,5,6]

    As an example
    If word_lists is [['a', 'b'], ['c','b']] then vocabulary should be {'a', 'b', 'c'} and word_to_index should be something like {'a':0, 'b':1, 'c':2} 
    (the permutation is irrelevant, this is just the obvious one.) 
    Also, index_to_word should be in this case {0:'a', 1:'b', 2:'c'}

    Args:
    word_lists: list of lists of strings

    Returns:
    """
    vocabulary = set()
    word_to_index = dict()
    index_to_word = dict()

    ### YOUR CODE 3-4 lines
    vocabulary = {x for word_list in word_lists for x in word_list}
    word_to_index = {x: i for (i, x) in enumerate(vocabulary)}
    index_to_word = {i: x for (i, x) in enumerate(vocabulary)}
    ### END CODE

    assert len(word_to_index) == len(index_to_word)
    return vocabulary, word_to_index, index_to_word
    
def words_to_vectors(word_lists, vocabulary, index_map):
    """
    Each list of strings in word_lists is turned into a vector of length |vocabulary| in such a way that the ith entry of the vector is the number of occurences of word i (under index_map) in the string.
    index_map is the dictionary {word : index}. If the word is not in the vocabulary you should ignore it. 

    As an example
    If the vocabulary is {'a','b','c','d','e'} and the mapping index_map is {'a':0,'b':1,'c':2,'d':3,'e':4}, then the word_list ['a','b','a'] is mapped to [2, 1, 0, 0, 0]

    A way of doing this is as follows (you can come up with a different implementation as long as the results are the same):
    1. Map each word to its index transforming list of words to list of indices
    2. Fill the vectors using these indices

    Step 1 can be done easily.
    For the second step the collections.Counter class may be useful: given a list it returns a dictionary-like object counting the ocurrences of each entry in the list. 
    As an example of this:
    c = Counter([1,1,2]) # -> Counter({1: 2, 2: 1})
    Now c.keys, c.values, c.items gives the list of indices and counts
    In [1]: list(c.keys())
    Out[1]: [1, 2]
    In [2]: list(c.values())
    Out[2]: [2, 1]

    Remember indexing in numpy:
    if x is a numpy array and ind a list of indices, then a[ind] indexes into the entries of x given in ind.

    Args:
    word_lists: list of lists of strings (each string is in self.vocabulary)

    Returns: 
    word_vectors: numpy array of size |word_lists| X |vocabulary|. The ith row is the vector to which the ith word_list is mapped by counting the occurrences of each word.
    """
    word_vectors = np.zeros((len(word_lists), len(vocabulary)))

    ### YOUR CODE
    index_lists = [[index_map[x] for x in word_list if x in index_map] for word_list in word_lists]
    #index_lists = [[index_map[x] for x in word_list] for word_list in word_lists]


    counters = [Counter(x) for x in index_lists]
    for i, c in enumerate(counters):
        indices = [x[0] for x in c.items()]
        counts = [x[1] for x in c.items()]
        word_vectors[i, indices] = counts
    ### END CODE

    return word_vectors

def data_clean(strings, stopwords = set()):
        """
        Clean the data by calling the string functions you made
        Nothing required here
        """
        clean_data = standardize_strings(strings)
        word_lists = split_strings(clean_data)
        return word_lists
    
def run_bow_classifier(train_strings, test_strings, train_labels, test_labels):
    """ Transform strings to vectors and run a logistic regression model on it and see the results 

    You can use the following Logisticregression model (ask google for details if needed). Supports fit and score
    from sklearn.linear_model import LogisticRegression

    1. transform data, both train and test (use same transform for both)
    2. fit Logistic Regression model
    3. Print train score and test score
    """
    print(train_strings[0:10])
    ### YOUR CODE HERE
    clean_train_strings = data_clean(train_strings)
    clean_test_strings = data_clean(test_strings)
    vocabulary, word_to_index, index_to_word = make_vocabulary_and_index(clean_train_strings)
    
    X_train = words_to_vectors(clean_train_strings, vocabulary, word_to_index)
    X_test = words_to_vectors(clean_test_strings, vocabulary, word_to_index)
    clf = LogisticRegression(random_state=0).fit(X_train, train_labels)
    train_score =  clf.score(X_train, train_labels)
    test_score = clf.score(X_test, test_labels)
    print(f'train mean accuracy {train_score}, test mean accuracy {test_score}')
    
    ### END CODE
  
test_string_cleaning()
test_vectorize()  
run_bow_classifier(*get_data())

TESTING 'standardize_strings': 			PASSED!
TESTING 'split_strings': 			PASSED!
TESTING 'make_vocabulary_and_index': 		PASSED!
TESTING 'words_to_vector': 			PASSED!
['Probably not, still going over some stuff here\n', 'I HAVE A DATE ON SUNDAY WITH WILL!!\n', 'Thanks 4 your continued support Your question this week will enter u in2 our draw 4 Â£100 cash. Name the NEW US President? txt ans to 80082\n', "Dear 0776xxxxxxx U've been invited to XCHAT. This is our final attempt to contact u! Txt CHAT to 86688 150p/MsgrcvdHG/Suite342/2Lands/Row/W1J6HL LDN 18yrs\n", 'I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones\n', 'Kothi print out marandratha.\n', 'Arun can u transfr me d amt\n', 'I asked you to call him now ok\n', 'Ringtone Club: Gr8 new polys direct to your mobile every week !\n', 'Hello! Just got here, st andrews-boy i

## Ex2: TFIDF Using standard tools 

See here for a default implementation of transforming text to TFIDF transformed vectors
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

Re do the text classification from above using the TfidfVectorizer class.
You can play with the parameters if you like.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


def run_tfidf_classifier(train_strings, test_strings, train_labels, test_labels):
    """ Transform strings to vectors and run a logistic regression model on it and see the results 

    You can use the following Logisticregression model (ask google for details if needed). Supports fit and score
    from sklearn.linear_model import LogisticRegression

    1. transform data, both train and test using TFIDF transform
    2. fit Logistic Regression model
    3. Print train score and test score
    """
    ### YOUR CODE HERE
    vectorizer = TfidfVectorizer()
    #vectorizer = CountVectorizer()
    #train_strings = standardize_strings(train_strings)
    #test_strings = standardize_strings(test_strings)

    vectorizer.fit(train_strings)
    X_train = vectorizer.transform(train_strings)
    X_test = vectorizer.transform(test_strings)
    clf = LogisticRegression(random_state=0)
    clf.fit(X_train, train_labels)
    train_score =  clf.score(X_train, train_labels)
    test_score = clf.score(X_test, test_labels)
    print(f'train mean accuracy {train_score}, test mean accuracy {test_score}')
    ### END CODE


run_tfidf_classifier(*get_data())

train mean accuracy 0.9734868773433315, test mean accuracy 0.967391304347826


## Ex3: Analyzing Feature Hashing
In feature hashing, we map a vector $x \in R^d$ to a vector in $R^k$ using two hash functions $h : [d] \to [k]$ and $g : [d] \to \{-1,1\}$. The hash functions are chosen randomly and independently before seeing any data. Assume the hash functions satisfy the following two properties:
1. For any two distinct coordinates $i \neq j$, we have that $g(i)$ and $g(j)$ are independent and uniform random, i.e. for any $a,b \in \{-1,1\}$ it holds that $\Pr_g[g(i)=a \wedge g(j)=b] = 1/4$.
2. For any two distinct coordinates $i \neq j$, we have that $\Pr_h[h(i)=h(j)] \leq 1/k$.

The embedding $f(x)$ of a vector $x$ is obtained by hashing each index $i \in [d]$ to the index $h(i)$ and adding $g(i) \cdot x_i$ to $f(x)_{h(i)}$.

Your task it to prove:
1. For two vectors $x,y$, we have $\mathbb{E}[f(x)^\intercal f(y)] = x^\intercal y$.

Hint: The following re-writing may be useful:
$$
f(x)^\intercal f(y) = \sum_{i=1}^d \sum_{j=1}^d 1_{[h(i)=h(j)]} x_i y_j g(i) g(j),
$$
where $1_{[h(i)=h(j)]}$ is the indicator random variable taking the value $1$ if $h(i)=h(j)$ and $0$ otherwise.
You may also need linearity of expectation $\mathbb{E}[A + B] = \mathbb{E}[A] + \mathbb{E}[B]$ and that for independent random variables $X,Y$ we have $\mathbb{E}[XY] = \mathbb{E}[X]\mathbb{E}[Y]$.

### BEGIN MATH SOLUTION
1. Using the hint and linearity of expectation, we see that
$$
\mathbb{E}[f(x)^\intercal f(y)] = \sum_{i=1}^d \sum_{j=1}^d \mathbb{E}[1_{[h(i)=h(j)]} x_i y_j g(i) g(j)]
$$
Using that $h$ and $g$ are chosen independently and that $g(i)$ is independent of $g(j)$ when $i \neq j$, it holds for any term with $i \neq j$ that $\mathbb{E}[1_{[h(i)=h(j)]} x_i y_j g(i) g(j)] = \mathbb{E}[1_{[h(i)=h(j)]}] x_i y_j \mathbb{E}[g(i)] \mathbb{E}[g(j)]$. Since $g(i)$ is uniform random among $\{-1,1\}$, its expectation is $0$ and the whole term is $0$. Therefore we have
$$
\mathbb{E}[f(x)^\intercal f(y)] = \sum_{i=1}^d \mathbb{E}[1_{[h(i)=h(i)]} x_i y_j g(i)^2]
$$
The indicator $1_{[h(i)=h(i)]}$ is always $1$ and $g(i)^2$ is also $1$. Hence
$$
\mathbb{E}[f(x)^\intercal f(y)] = \sum_{i=1}^d x_i y_i.
$$
### END SOLUTION


## Ex4: Skip-Gram
This exercise can be found in the separate notebook skipgram.ipynb. We recommend uploading the notebook to Google Colab for faster execution.